# Exercise Set 8

Due: **9:30 30 May 2022**

Discussion: **13:00 3 June 2022**

**Online submission** at via [ILIAS](https://www.ilias.uni-koeln.de/ilias/goto_uk_exc_4593683.html) in the directory Exercises / Übungen -> Submission of Exercises / Rückgabe des Übungsblätter

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import astropy.constants as const
import astropy.units as u
import pandas as pd

# 1. Error Calculations [40 points]

Suppose we are viewing a binary orbit face-on. The primary star has mass $2.19^{+0.43}_{-0.41} M_\odot$, luminosity $60.8^{+1.3}_{-1.2} L_\odot$, and effective temperature $6595^{+53}_{-58} K$. The secondary star has mass $1.62^{+0.26}_{-0.32} M_\odot$, luminosity $3.2^{+0.7}_{-0.8} L_\odot$, and effective temperature $4284^{+78}_{-73} K$. The distance to the binary is determined to be $5pc$. Compute the following properties including error.

 > *hint: recall the solar values (without error)*
 
 >> $M_\odot = 1.989 \times 10^{30} kg$, 
 
 >> $m_\odot = -26.74$, 
 
 >> $L_\odot=3.828 \times 10^{26} \frac{J}{s}$, 
 
 >> $R_\odot = 6.955 \times 10^{8} m$, and 
 
 >> $T_{eff, \odot} = 5780 K$

**a)** What is the total mass of the binary? What is the reduced mass? **10 points**

 > *hint: the reduced mass of $M_1$ and $M_2$ is $\mu = \frac{M_1M_2}{M_1+M_2}$*

In [10]:
#Mass1 and errors
M1=2.19
M1_u=0.43
M1_l=0.41
#Mass2 and errors
M2=1.62
M2_u=0.26
M2_l=0.32
#Total mass and errors
Mtot=M1+M2
Mtot_u=np.sqrt(M1_u**2+M2_u**2)
Mtot_l=np.sqrt(M1_l**2+M2_l**2)

print("Total mass:",Mtot,"(+",np.round(Mtot_u,2),",-",np.round(Mtot_l,2),")")

#Reduced mass
def reduced_mass(M1,M2,M1_u,M1_l,M2_u,M2_l):
    Mtot=M1+M2
    Mtot_u=np.sqrt(M1_u**2+M2_u**2)
    Mtot_l=np.sqrt(M1_l**2+M2_l**2) 
    #calculate reduced mass
    mu=M1*M2/Mtot
    #upper bound
    mu_u=abs(mu)*np.sqrt((M1_u/M1)**2+(M2_u/M2)**2)
    #lower bound
    mu_l=abs(mu)*np.sqrt((M1_l/M1)**2+(M2_l/M2)**2)
    print("Reduced mass:",np.round(mu,2),"(+",np.round(mu_u,2),",-",np.round(mu_l,2),")")
    return mu,mu_u,mu_l

mu=reduced_mass(M1,M2,M1_u,M1_l,M2_u,M2_l)
#print("Reduced mass:",np.round(mu[0],2),"+",np.round(mu[1],2),"-",np.round(mu[2],2))

Total mass: 3.81 (+ 0.5 ,- 0.52 )
Reduced mass: 0.93 (+ 0.24 ,- 0.25 )


**b)** What is the radius of each star? **10 points**

 > *hint: recall the equation for the luminosity of a star $L = 4\pi R^2 \sigma T_{eff}^4$, where $\sigma$ is the Stefan-Boltzmann constant*

In [37]:
L1=(60.8*((u.L_sun).to("J/s")))
L1_u=(1.3*((u.L_sun).to("J/s")))
L1_l=(1.2*((u.L_sun).to("J/s")))
T1,T1_u,T1_l=[6595,53,58]
L2=(3.2*(u.L_sun).to("J/s"))
L2_u=(0.7*(u.L_sun).to("J/s"))
L2_l=(0.8*(u.L_sun).to("J/s"))
T2,T2_u,T2_l=[4284,78,73]
def rad_err(L,dL,T,dT):
    sigma=const.sigma_sb.value # Steffan boltz in SI units
    T4=T**4
    T5=T4*T
    term1=0.5*((L/(4*np.pi*sigma*T4))**(-1.5))*(1/(4*np.pi*sigma*T4))
    term2=0.5*((L/(4*np.pi*sigma*T4))**(-1.5))*(-L/(np.pi*sigma*T5))
    dr=np.sqrt((term1*dL)**2 + (term2*dT)**2)
    return dr

def radius(L,L_u,L_l,T,T_u,T_l):
    sigma=const.sigma_sb.value # Steffan boltz in SI units
    T4=T**4
    R=(np.sqrt(L/(4*np.pi*sigma*T4))*(u.m)).to("R_sun").value
    R_u=(rad_err(L,L_u,T,T_u)*(u.m)).to("R_sun").value
    R_l=(rad_err(L,L_l,T,T_l)*(u.m)).to("R_sun").value
    print("Radius:",R,"(+",R_u,",-",R_l,")")
    return R,R_u,R_l

print("primary star:")
rad1=radius(L1,L1_u,L1_l,T1,T1_u,T1_l)
print("secondary star:")
rad2=radius(L2,L2_u,L2_l,T2,T2_u,T2_l)

primary star:
Radius: 5.97276394217297 (+ 6.6775628685287116e-21 ,- 6.976708790728586e-21 )
secondary star:
Radius: 3.2473509437426613 (+ 7.334516125467876e-20 ,- 8.243400780953687e-20 )


**c)** What is the flux coming from each star? What is the total flux? What is the apparent magnitude of the binary system? **20 points**

 > *hint: the flux is determined by $F = \sigma T_{eff}^4$, while apparant magnitude is given by $m = -2.5 log_{10}\left( \frac{F}{F_\odot} \right) + m_\odot$*

In [39]:
def flux_err(T,dT):
    sigma=const.sigma_sb.value # Steffan boltz in SI units
    T3=T**3
    df=4*sigma*T3*dT
    return df

#Returns Flux and errors in W/m^2
def flux(T,T_u,T_l):
    sigma=const.sigma_sb.value # Steffan boltz in SI units
    T4=T**4
    F=sigma*T4
    F_u=flux_err(T,T_u)
    F_l=flux_err(T,T_l)
    print("Flux:",F,"(+",F_u,",-",F_l,")")
    return F,F_u,F_l

print("primary star:")
F1=flux(T1,T1_u,T1_l)
print("secondary star:")
F2=flux(T2,T2_u,T2_l)

primary star:
Flux: 107268185.94755198 (+ 3448196.424697653 ,- 3773497.9741974315 )
secondary star:
Flux: 19098954.112451408 (+ 1390960.2434838563 ,- 1301796.1253118142 )


In [45]:
# apparent magnitude
def app_mag(F,F_u,F_l):
    T_sun=5780
    F_sun=flux(T_sun,0,0)[0]
    sun_mag=-26.74
    m=-2.5*(np.log10(F/F_sun))+sun_mag
    m_u=(-2.5*(np.log10(F_u/F_sun))+sun_mag)+m
    m_l=m+(2.5*(np.log10(F_l/F_sun))+sun_mag)
    print("Apparent magnitude:",m,"(+",m_u,",-",m_l,")")
    return m,m_u,m_l

print("primary star:")
F1=app_mag(F1[0],F1[1],F1[2])
print("secondary star:")
F2=app_mag(F2[0],F2[1],F2[2])

primary star:
Flux: 63288250.47661096 (+ 0.0 ,- 0.0 )
Apparent magnitude: nan (+ nan ,- nan )
secondary star:
Flux: 63288250.47661096 (+ 0.0 ,- 0.0 )
Apparent magnitude: nan (+ nan ,- nan )


# 2. PCA using covariance [60 points]

In this problem we will redo the Principal Component Analysis (PCA) as presented by Francis & Wills (1999) on a set of quasar data. The paper can be downloaded from this [link](https://arxiv.org/abs/astro-ph/9905079.pdf). 
The data is available on the website as datafile: `quasar.dat`. The main result from the paper is shown in their Table 3:

![Table 3 from Francis and Wills (1999)](table_3.png)

We now carry through a PCA on the data of the quasar sample given in Francis & Wills (1999).

**a)** Read the paper! Remove data rows that have missing data. Create a table of the original data and compute mean value and standard deviation of each column. **10 Points**

In [119]:
df=pd.read_csv("quasar.dat",sep="\s+",header=4,skipfooter=9,names=['PG Name', 'log L1216', 'alpha', 'logFWHM Hbeta ', 'FeII/ Hbeta ',
 'logEW [OIII]', 'logFWHM CIII]','logEW Lalpha', 'logEW CIV', 'CIV/ Lalpha', 'logEW CIII]', 'SiIII/ CIII]', 'NV/ Lalpha', '1400A/ Lalpha']  )
for i in range(len(df.columns)):
    col=df.columns[i]
    df=df[df[col]!="----"]
df
print(df)

#create a table of the original data and compute the mean value and standard deviation of each column

     PG Name  log L1216 alpha  logFWHM Hbeta   FeII/ Hbeta   logEW [OIII]  \
0   0953+414      45.83  1.57           3.496          0.25          1.26   
2   1114+445      44.99  0.88           3.660          0.20          1.23   
3   1115+407      45.41  1.89           3.236          0.54          0.78   
4   1116+215      46.00  1.73           3.465          0.47          1.00   
5   1202+281      44.77  1.22           3.703          0.29          1.56   
6   1216+069      46.03  1.36           3.715          0.20          1.00   
7   1226+023      46.74  0.94           3.547          0.57          0.70   
8   1309+355      45.55  1.51           3.468          0.28          1.28   
9   1322+659      45.42  1.69           3.446          0.59          0.90   
10  1352+183      45.34  1.52           3.556          0.46          1.00   
11  1402+261      45.74  1.93           3.281          1.23          0.30   
13  1415+451      45.08  1.74           3.418          1.25          0.30   

<ipython-input-119-077e419bea9f>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df=pd.read_csv("quasar.dat",sep="\s+",header=4,skipfooter=9,names=['PG Name', 'log L1216', 'alpha', 'logFWHM Hbeta ', 'FeII/ Hbeta ',


**b)** Take the original data and put it into normalized or weighted form, so that the effect of different units is effectively removed. Normalize by the standard deviation! **10 Points**

**c)** Visually inspect the data after the normalization by plotting each column ($x$: data index, $y$: data value). Confirm (by eye) that each component is about normally distributed. **10 Points**

**d)** Construct the covariance matrix. This is a $13\times13$ symmetric matrix. **10 Points**

$$ C_{ij} = \sigma_i \sigma_j $$

**e)** Compute the eigenvalues and eigenvectors of the covariance matrix. Plot the Eigenvalues against their number (index). Recreate Table 3 from Francis & Wills (1999). **10 Points**

**f)** Compute errors of the eigenvalues with a bootstrap analysis or jackknife. Use sample size of 10000. Plot the distributions for the first 5 eigenvalues. **10 Points**